# **Creating a Digital Twin**

In [ ]:
import copy

Now that we have a working Jupyter notebook installation and a basic familiarity with Python, we can begin creating the Digital Twin.  

In [1]:
genesis = {
    "AMM":{"A1":100,"A2":100,"s":100,"fee":0.01},
    "Trader":{"A1":100,"A2":100,"s":0},
    "LP":{"A1":0,"A2":0,"s":100}
    }

print("The amount of A1 in the AMM is:", genesis["AMM"]["A1"]);
print("The amount of A1 of the agent is:", genesis["Trader"]["A1"]);
print("The amount of A2 of the AMM is:", genesis["AMM"]["A2"]);
print("The amount of A2 of the agent is:", genesis["Trader"]["A2"]);

The amount of A1 in the AMM is: 100
The amount of A1 of the agent is: 100
The amount of A2 of the AMM is: 100
The amount of A2 of the agent is: 100


In [ ]:
def swapToAsset2(state, inputs):
    """Swaps a specified amount of asset 1 (A1) from an agent to asset 2 (A2) in the AMM.

    Args:
        state: A dictionary containing the current state of the system, including
              the AMM pool information and the agent's asset balances.
        inputs: A list containing two elements:
            - agent: The name of the agent who wants to swap.
            - dA1: The amount of asset 1 the agent wants to swap.

    Returns:
        None. The function updates the state dictionary in-place.

    Raises:
        ValueError: If the agent does not have enough A1 to perform the swap.
    """

    # Get the agent and the amount of A1 to swap from the inputs.
    agent = inputs[0];
    dA1 = inputs[1];

    # Calculate the fee factor based on the AMM fee.
    feeFactor = (1 - state["AMM"]["fee"]);

    # Calculate the amount of A2 received by the agent after considering the fee.
    dA2 = (state["AMM"]["A2"] * dA1 * feeFactor) / (state["AMM"]["A1"] + dA1 * feeFactor);

    # Check if the agent has enough A1 to perform the swap.
    if dA1 > 0 and state[agent]["A1"] - dA1 >= 0:
        # If the agent has enough A1, update the state accordingly.
        state["AMM"]["A1"] += dA1;  # Add dA1 to the AMM's A1 pool
        state[agent]["A1"] -= dA1;  # Remove dA1 from the agent's A1 balance
        state["AMM"]["A2"] -= dA2;  # Remove dA2 from the AMM's A2 pool
        state[agent]["A2"] += dA2;  # Add dA2 to the agent's A2 balance
        print("Amount of A2 that the agent recieves after considering the fee:", dA2);
    else:
        # If the agent does not have enough A1, raise an error.
        raise ValueError("Agent does not have enough A1 to perform the swap.");

inputs = ["Trader",50];
agent = "Trader";

swapToAsset2(genesis, inputs);

print("The amount of A1 in the AMM is:", genesis["AMM"]["A1"]);
print("The amount of A1 of the agent is:", genesis[agent]["A1"]);
print("The amount of A2 of the AMM is:", genesis["AMM"]["A2"]);
print("The amount of A2 of the agent is:", genesis[agent]["A2"]);

Amount of A2 that the agent recieves after considering the fee: 63.79434203531559
The amount of A1 in the AMM is: 116.8896321070234
The amount of A1 of the agent is: 83.1103678929766
The amount of A2 of the AMM is: 86.2056579646844
The amount of A2 of the agent is: 113.7943420353156


In [ ]:
def swapToAsset1(state, inputs):
    """Swaps a specified amount of asset 2 (A2) from an agent to asset 1 (A1) in the AMM.

    Args:
        state: A dictionary containing the current state of the system, including
              the AMM pool information and the agent's asset balances.
        inputs: A list containing two elements:
            - agent: The name of the agent who wants to swap.
            - dA2: The amount of asset 2 the agent wants to swap.

    Returns:
        None. The function updates the state dictionary in-place.

    Raises:
        ValueError: If the agent does not have enough A1 to perform the swap.
    """

    # Get the agent and the amount of A2 to swap from the inputs.
    agent = inputs[0];
    dA2 = inputs[1];

    # Calculate the fee factor based on the AMM fee.
    feeFactor = (1 - state["AMM"]["fee"]);

    # Calculate the amount of A1 received by the agent after considering the fee.
    dA1 = (state["AMM"]["A1"]* dA2 * feeFactor) / (state["AMM"]["A2"] + dA2 * feeFactor);

    # Check if the agent has enough A2 to perform the swap.
    if dA2 > 0 and state[agent]["A2"] - dA2 >= 0:
        # If the agent has enough A2, update the state accordingly.
        state["AMM"]["A2"] += dA2;  # Add dA2 to the AMM's A2 pool
        state[agent]["A2"] -= dA2;  # Remove dA2 from the agent's A2 balance
        state["AMM"]["A1"] -= dA1;  # Remove dA1 from the AMM's A1 pool
        state[agent]["A1"] += dA1;  # Add dA1 to the agent's A1 balance
        print("Amount of A1 that the agent recieves after considering the fee:", dA1);
    else:
        # If the agent does not have enough A2, raise an error.
        raise ValueError("Agent does not have enough A2 to perform the swap.");

inputs = ["Trader",50];
agent = "Trader";

swapToAsset1(genesis, inputs);

print("The amount of A1 in the AMM is:", genesis["AMM"]["A1"]);
print("The amount of A1 of the agent is:", genesis[agent]["A1"]);
print("The amount of A2 of the AMM is:", genesis["AMM"]["A2"]);
print("The amount of A2 of the agent is:", genesis[agent]["A2"]);

Amount of A1 that the agent recieves after considering the fee: 42.6366658257049
The amount of A1 in the AMM is: 74.25296628131849
The amount of A1 of the agent is: 125.74703371868151
The amount of A2 of the AMM is: 136.2056579646844
The amount of A2 of the agent is: 63.794342035315594


In [ ]:
def addLiquidity(state, inputs):
    """Adds liquidity to the AMM pool by an agent depositing assets.

    Args:
      state: A dictionary containing the current state of the system, including
             the AMM pool information (A1, A2, total liquidity (s)) and the agent's asset balances.
      inputs: A list containing three elements:
          - agent: The name of the agent who wants to add liquidity.
          - dA1: The desired amount of asset 1 the agent wants to deposit.
          - dA2: The desired amount of asset 2 the agent wants to deposit.

    Returns:
        None. The function updates the state dictionary in-place.

    Raises:
        ValueError: If the agent does not have enough of either asset to fulfill
                    the minimum deposit requirement based on the current pool ratio.
    """

    # Get agent name and desired deposit amounts for each asset.
    agent = inputs[0];

    # Get current AMM pool information.
    A1 = state["AMM"]["A1"]; # Current amount of asset 1 in the pool
    A2 = state["AMM"]["A2"]; # Current amount of asset 2 in the pool
    S = state["AMM"]["s"]; # Current total liquidity of the AMM pool

    # Calculate the minimum deposit amount of each asset based on the current pool ratio.
    # This ensures a proportional contribution to maintain the pool's price ratio.
    dA1 = min(inputs[1], A1/A2*inputs[2]);
    dA2 = min(inputs[2], A2/A1*inputs[1]);

    # Check if the agent has enough of both assets to meet the minimum deposit requirement.
    if(dA1<=state[agent]["A1"] and dA2 <= state[agent]["A2"]) and (dA1 > 0 and dA2 >0):
      # If enough assets, update the agent's and AMM pool's balances.
      state[agent]["A1"] -= dA1  # Remove dA1 from agent's A1 balance
      state[agent]["A2"] -= dA2  # Remove dA2 from agent's A2 balance
      state["AMM"]["A1"] += dA1  # Add dA1 to the AMM's A1 pool
      state["AMM"]["A2"] += dA2  # Add dA2 to the AMM's A2 pool

      # Calculate the change in total liquidity based on the minimum deposit proportion.
      dS= min(dA1/A1, dA2/A2) * S;
      # Update the total liquidity of the AMM pool and the agent's share.
      state["AMM"]["s"] += dS  # Increase total liquidity by dS
      state[agent]["s"] += dS  # Increase agent's liquidity share by dS

    else:
    # If agent doesn't have enough assets, raise an error.
      raise ValueError("Agent does not have enough assets to meet minimum deposit requirement.")


In [ ]:
def removeLiquidity(state,inputs):
  # Get agent name and the desired amount of liquidity to remove (dS).
  agent = inputs[0];
  dS = inputs[1];

  # Check if requested removal is positive, within agent's liquidity share, and
  # doesn't exceed total pool liquidity.
  if(dS>0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0):
    # Calculate the proportion of AMM pool balances to be withdrawn based on dS.
    DA = (1-dS/state["AMM"]["s"]);
    # Get current AMM pool balances.
    A1 = state["AMM"]["A1"];
    A2 = state["AMM"]["A2"];
    # Update agent's liquidity share.
    state[agent]["s"]-=dS;
    # Update AMM pool balances by applying the withdrawal factor.
    state["AMM"]["A1"]=A1*DA;
    state["AMM"]["A2"]=A2*DA;

    # Calculate the amount of each asset the agent receives based on the difference
    # between original pool balances and the updated balances.
    state[agent]["A1"]+= A1-state["AMM"]["A1"];
    state[agent]["A2"]+= A2-state["AMM"]["A2"];
    # Update total liquidity of the AMM pool.
    state["AMM"]["s"]-=dS;
  else:
    # If agent doesn't have enough assets, raise an error (not shown here).
    raise ValueError("Agent does not have enough assets to meet minimum removal requirement.")
